In [ ]:
# Sign Language Recognition - Model Training

This notebook demonstrates the process of training a sign language recognition model using landmarks extracted from MediaPipe Hands.

We'll explore different classifiers and hyperparameter tuning to achieve the best performance.


In [ ]:
## 1. Import Libraries


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

# Add parent directory to path so we can import our modules
sys.path.append('..')
from utils import load_landmarks_from_csv, plot_confusion_matrix

# Set random seed for reproducibility
np.random.seed(42)


In [ ]:
## 2. Load and Explore Data

We'll load the landmark data that was collected using our data collection tool.


In [ ]:
# Path to the landmarks CSV file
landmarks_file = '../data/landmarks/landmarks_20231015_120000.csv'  # Update this path

# Check if the file exists
if not os.path.exists(landmarks_file):
    print(f"File not found: {landmarks_file}")
    print("Please collect data first using the data_collection.py script.")
else:
    # Load the data
    X, y = load_landmarks_from_csv(landmarks_file)
    print(f"Data loaded successfully! X shape: {X.shape}, y shape: {y.shape}")


In [ ]:
### 2.1 Explore Class Distribution


In [ ]:
# Get class distribution
classes, counts = np.unique(y, return_counts=True)
class_distribution = pd.DataFrame({'Class': classes, 'Count': counts})

# Sort by class
class_distribution = class_distribution.sort_values('Class')

# Plot class distribution
plt.figure(figsize=(12, 6))
sns.barplot(x='Class', y='Count', data=class_distribution)
plt.title('Class Distribution')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Print summary
print(f"Total samples: {len(y)}")
print(f"Number of classes: {len(classes)}")
print(f"Classes: {', '.join(classes)}")


In [ ]:
## 3. Preprocess Data


In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")


In [ ]:
## 4. Train a Random Forest Classifier

We'll start with a simple Random Forest classifier with default parameters.


In [ ]:
# Initialize the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
%time rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
### 4.1 Evaluate Model Performance


In [ ]:
# Print classification report
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Plot confusion matrix
plt.figure(figsize=(12, 10))
plot_confusion_matrix(
    y_test,
    y_pred,
    label_encoder.classes_,
    normalize=True,
    title='Normalized Confusion Matrix'
)
plt.tight_layout()
plt.show()
